In [1]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

C:\Users\Dark\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

295 documents
135 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'bridge-useful', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-1o', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-2o', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-3o', 'fact-4o', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bull

In [6]:

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [8]:

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.asarray(training, dtype="object")
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


C:\Users\Dark\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/200
59/59 [==============================] - 0s 1ms/step - loss: 4.9202 - accuracy: 0.0169
Epoch 2/200
59/59 [==============================] - 0s 1ms/step - loss: 4.8303 - accuracy: 0.0339
Epoch 3/200
59/59 [==============================] - 0s 1ms/step - loss: 4.7657 - accuracy: 0.0610
Epoch 4/200
59/59 [==============================] - 0s 1ms/step - loss: 4.6409 - accuracy: 0.0610
Epoch 5/200
59/59 [==============================] - 0s 1ms/step - loss: 4.5283 - accuracy: 0.0542
Epoch 6/200
59/59 [==============================] - 0s 1ms/step - loss: 4.4484 - accuracy: 0.0610
Epoch 7/200
59/59 [==============================] - 0s 1ms/step - loss: 4.2806 - accuracy: 0.0814
Epoch 8/200
59/59 [==============================] - 0s 1ms/step - loss: 4.1559 - accuracy: 0.1220
Epoch 9/200
59/59 [==============================] - 0s 1ms/step - loss: 4.0328 - accuracy: 0.0983
Epoch 10/200
59/59 [==============================] - 0s 1ms/step - loss: 3.8701 - accuracy: 0.1153
Epoch 11/

59/59 [==============================] - 0s 1ms/step - loss: 0.7814 - accuracy: 0.7356
Epoch 83/200
59/59 [==============================] - 0s 1ms/step - loss: 0.9671 - accuracy: 0.6915
Epoch 84/200
59/59 [==============================] - 0s 1ms/step - loss: 0.8256 - accuracy: 0.7492
Epoch 85/200
59/59 [==============================] - 0s 1ms/step - loss: 1.0073 - accuracy: 0.6881
Epoch 86/200
59/59 [==============================] - 0s 1ms/step - loss: 0.9222 - accuracy: 0.7119
Epoch 87/200
59/59 [==============================] - 0s 1ms/step - loss: 0.8097 - accuracy: 0.7593
Epoch 88/200
59/59 [==============================] - 0s 1ms/step - loss: 0.8306 - accuracy: 0.7559
Epoch 89/200
59/59 [==============================] - 0s 1ms/step - loss: 0.8812 - accuracy: 0.7119
Epoch 90/200
59/59 [==============================] - 0s 1ms/step - loss: 0.9585 - accuracy: 0.7051
Epoch 91/200
59/59 [==============================] - 0s 1ms/step - loss: 0.8805 - accuracy: 0.7424
Epoch 92/200


59/59 [==============================] - 0s 1ms/step - loss: 0.7862 - accuracy: 0.7559
Epoch 164/200
59/59 [==============================] - 0s 1ms/step - loss: 0.8043 - accuracy: 0.7627
Epoch 165/200
59/59 [==============================] - 0s 1ms/step - loss: 0.7334 - accuracy: 0.7559
Epoch 166/200
59/59 [==============================] - 0s 1ms/step - loss: 0.6674 - accuracy: 0.8000
Epoch 167/200
59/59 [==============================] - 0s 1ms/step - loss: 0.6610 - accuracy: 0.7932
Epoch 168/200
59/59 [==============================] - 0s 1ms/step - loss: 0.7329 - accuracy: 0.7932
Epoch 169/200
59/59 [==============================] - 0s 1ms/step - loss: 0.7670 - accuracy: 0.7559
Epoch 170/200
59/59 [==============================] - 0s 1ms/step - loss: 0.7656 - accuracy: 0.7797
Epoch 171/200
59/59 [==============================] - 0s 996us/step - loss: 0.8306 - accuracy: 0.7424
Epoch 172/200
59/59 [==============================] - 0s 984us/step - loss: 0.8269 - accuracy: 0.7288


In [12]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Dark\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Dark\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Dark\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Dark\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Dark\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\Dark\AppData\Roaming\nltk_dat

True

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [15]:
while True:
    input_data = input("You- ")
    if(input_data == "exit"):
        break
    answer = chatbot_response(input_data)
    answer

You- hello
1/1 [==============================] - 0s 16ms/step
You- hello
1/1 [==============================] - 0s 17ms/step
You- hello
1/1 [==============================] - 0s 17ms/step
You- exit
